In [ ]:
# Had to run from the shell:
# ipython kernel install --name "local-conda-env" --user
#import required libraries
from azure.ml import MLClient
from azure.ml.entities import ComponentJob, Code, PipelineJob, Dataset, InputDatasetEntry

In [ ]:
#Enter details of your AML workspace
subscription_id = '589c7ae9-223e-45e3-a191-98433e0821a9'
resource_group = 'amlisdkv2-rg-1635933774'
workspace = 'amlisdkv21635933774'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

In [ ]:
train_data = Dataset(
    paths=['azureml:Boston_Train_PQ:1635933774']
    # name="Boston_Train_PQ",
    # version="1635933774"
)

In [ ]:
pipeline_inputs = { 
    'target_column_name':'y',
    'my_training_data': 'azureml:Boston_Train_PQ:1635933774'
}

In [ ]:
train_job_inputs = {
    'target_column_name': '${{inputs.target_column_name}}',
    'training_data': '${{inputs.my_training_data}}',
    'categorical_features': '[]',
    'continuous_features': '["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE","DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"]'
}
train_job_outputs = {
    'trained_model': None
}

train_job = ComponentJob(
    component="TrainBostonForRAI:1635933774",
    inputs = train_job_inputs,
    outputs=train_job_outputs
)

In [ ]:
pipeline_job = PipelineJob(
    description="Notebook submitted Boston",
    jobs = {
        'train-model-job': train_job
    },
    inputs=pipeline_inputs,
    outputs=train_job_outputs,
    compute="cpucluster"
)

In [ ]:
#submit the pipeline job
returned_job = ml_client.jobs.create_or_update(pipeline_job)
#get a URL for the status of the job
returned_job.services["Studio"].endpoint